In [1]:
import numpy as np
from math import erf
from scipy import optimize
from scipy import special

C:\Users\00103168\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\00103168\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\00103168\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Input parameters
# Gas constant
R=8.3145
# Reference pressure
Pr=1.5e9
# Reference temperature
TKr=1473.
T0=273.
# Grain size and reference grain size
d=1.e-3
dr=1.e-3
# Reference density
rhor=3176.15
# Thermal expansivity
alphaT=3.59e-5
# Bulk modulus
bmod=115.2
# Raw frequency
freq=0.01

# Brent temperature minimization bounds
AX=0.
CX=2000.
tol=1e-3

# Other parameters (density, compressibility etc.):
rho0=3300.
a0=2.832e-5
a1=0.758e-8
K0=130.e9
KT=4.8
grun=6.
p0=3330.
AY=1.0
CY=1.6

# Set key parameters
Ab=0.664
alpha=0.38
tauP=6.e-5
Teta=0.94
beta=0.
delphi=0.
gamma=5.
lambdaphi=0.

# Set dV0 vs. P parameterisation parameters
y_a = -7.334963115431564676e-23
y_b = 7.510867653681621105e-12
y_c = 1.000184023114681908e+00

# Set solidus temperature at 50 km depth
solidus_50km = 1380
def funcVs(T,Vs_obs,m,dep,Mg):

  # Difference between observed and calculated temperature
  funcVs=np.abs(Vs_obs-Vs_calc(m,T,dep,Mg))
  return funcVs

def funcV0(x,Pin,K0,KT):

  # Difference between observed and calculated pressure for given value of P
  funcV0=np.abs(((K0*(3./2.)*(x**(7./3.)-x**(5./3.))*(1.+(((3./4.)*(KT-4.))*(x**(2./3.)-1.))))*1e-9)-(Pin*1e-9))
  return funcV0


In [3]:
#ALL 1333
pos=np.array([ 7.51212501e+01, -1.28055870e-02,  1.96132180e+00,  2.53289695e+01,
        8.59644885e+05,  2.99627780e-06,  1.14668689e+00, -5.14921992e+00,
       -1.64919976e+03])

In [4]:
# Inport data
tomo_NA_stack = np.loadtxt('ANT20_grid_PriorInversion.txt')

In [5]:
def Vs_calc3(m,T,dep,Mg):

  mu0 = m[0]
  dmudT = m[1]
  dmudP = m[2]
  eta0 = 10**m[3]
  E = m[4]
  Va = m[5]
  dTdz = m[6]
  dmudMg = m[7]
  dpdMg = m[8]
  sol50 = 1380

  Pg=(dep/30.)
  P=Pg*1.e9
  TK=T+273.

  Tsol=sol50+(dTdz*(dep-50.))
  Tn=TK/(Tsol+273.)

  Aeta=(1./gamma)*np.exp(-delphi)*np.ones(len(Tn))
  Aeta[Tn < Teta] = 1
  Aeta[(Tn < Teta) & (Tn <1)] = np.exp((-1.*((Tn[(Tn < Teta) & (Tn <1)]-Teta)/(Tn[(Tn < Teta) & (Tn <1)]-(Tn[(Tn < Teta) & (Tn <1)]*Teta))))*np.log(gamma))


    
  # Work out viscosity given A
  eta=((eta0*np.exp((E/R)*(1./TK-1./TKr))*np.exp((Va/R)*(P/TK-Pr/TKr)))*Aeta)

  # Unrelaxed compliance
  Ju=1./(1.e9*(mu0+(dmudP*Pg)+(dmudT*T)+(dmudMg*(100-Mg)/100)))


  # Determine input parameters for complex compliance terms
  tauM=eta*Ju
  tau=(3.*dep)/4.2
  tauS=tau/(2*np.pi*tauM)

  Ap=(0.03+beta)*np.ones(len(Tn))

  Ap [Tn < 0.91]=0.01
  Ap [(Tn>=0.91)&(Tn<0.96)]=0.01+(0.4*(Tn[(Tn>=0.91)&(Tn<0.96)]-0.91))
  Ap [(Tn>=0.96)&(Tn<1)]=0.03

  sigmap=7*np.ones(len(Tn))

  sigmap [Tn < 0.92]=4
  sigmap [(Tn>=0.92)&(Tn<1)]=4.+(37.5*(Tn[(Tn>=0.92)&(Tn<1)]-0.92))

  # Determine complex compliance terms
  J1=Ju*(1.+((Ab*(tauS**alpha))/alpha)+((np.sqrt(2.*np.pi)/2.)*Ap*sigmap*(1.-special.erf((np.log(tauP/tauS))/(np.sqrt(2.)*sigmap)))))

  # include pressure and temperature-dependent alpha
  dV0=optimize.brent(funcV0,brack=(AY,CY),args=(P,K0,KT,),tol=tol)
  alphaP0=dV0*np.exp((grun+1.)*((dV0**(-1.))-1.))
  rhoP0=(p0+dpdMg*(Mg-89.27)/100)*dV0
  intalphaT=(a0*(TK-273.))+((a1/2.)*((TK**2.)-(273.**2.)))
  rho=(rhoP0)*(1.-(alphaP0*intalphaT))

  # Calculate Vs
  Vs=1./(np.sqrt(rho*J1)*1000.)  
  TableVRho=np.empty([nT, 4])
  TableVRho[:, 0] = Vs
  TableVRho[:, 1] = rho
  TableVRho[:, 2] = Mg
  TableVRho[:, 3] = eta
  return TableVRho

In [14]:
Tmin = 300-273.
Tmax = 3000-273.
dT = 1.      # temperature increment
nT = int((Tmax - Tmin)/dT + 1)
depth=tomo_NA_stack[:,2]
Depths=np.unique(depth)
nDepths = Depths.shape[0]

In [15]:
Mgt = np.arange(89.27,93.97,0.1)

In [16]:
nMg=Mgt.shape[0]

In [17]:
Mgt

array([89.27, 89.37, 89.47, 89.57, 89.67, 89.77, 89.87, 89.97, 90.07,
       90.17, 90.27, 90.37, 90.47, 90.57, 90.67, 90.77, 90.87, 90.97,
       91.07, 91.17, 91.27, 91.37, 91.47, 91.57, 91.67, 91.77, 91.87,
       91.97, 92.07, 92.17, 92.27, 92.37, 92.47, 92.57, 92.67, 92.77,
       92.87, 92.97, 93.07, 93.17, 93.27, 93.37, 93.47, 93.57, 93.67,
       93.77, 93.87, 93.97])

In [18]:
Depths

array([-400000., -390000., -380000., -370000., -360000., -350000.,
       -340000., -330000., -320000., -310000., -300000., -290000.,
       -280000., -270000., -260000., -250000., -240000., -230000.,
       -220000., -210000., -200000., -190000., -180000., -170000.,
       -160000., -150000., -140000., -130000., -120000., -110000.,
       -100000.,  -90000.,  -80000.,  -70000.,  -60000.,  -50000.,
        -40000.,  -30000.,  -20000.,  -10000.])

In [19]:
Result_T_F = list()
Result_Rho_F = list()
Result_eta_F = list()
Result_V_F = list()
Result_Mg_F = list()

arr_T = np.linspace(Tmin, Tmax, nT)
for j in range(nDepths):
    Result_T = np.array([]).reshape(0,661*661)
    Result_Rho = np.array([]).reshape(0,661*661)
    Result_eta = np.array([]).reshape(0,661*661)
    Result_V = np.array([]).reshape(0,661*661)
    Result_Mg = np.array([]).reshape(0,661*661)
    arr_select = tomo_NA_stack[:, 2]== Depths[j]
    obs_v = tomo_NA_stack[:, 3][arr_select]/1e3
    obs_den = tomo_NA_stack[:, 4][arr_select]

    for i in range(nMg):
        arr_TVRho = np.zeros([nT, 5])
        arr_TVRho[:, 0] = arr_T
        arr_TVRho[:, 1:] = Vs_calc3(pos,arr_T,-Depths[j]/1e3,Mgt[i])
        syn_T = arr_TVRho[:, 0]
        syn_v = arr_TVRho[:, 1]
        syn_mg = arr_TVRho[:, 3] 
        syn_rho = arr_TVRho[:, 2]  
        syn_eta = arr_TVRho[:, 4]
        ipkwds = dict(left=-1, right=-1)
        result_T = np.interp(obs_v, syn_v[::-1], syn_T[::-1], **ipkwds)
        result_rho = np.interp(obs_v, syn_v[::-1], syn_rho[::-1], **ipkwds)
        result_eta = np.interp(obs_v, syn_v[::-1], syn_eta[::-1], **ipkwds)
        result_v = np.interp(obs_v, syn_v[::-1], syn_v[::-1], **ipkwds)
        result_mg = result_rho - result_rho + Mgt[i]

        Result_V = np.vstack((Result_V, result_v))
        Result_T = np.vstack((Result_T, result_T))
        Result_Rho = np.vstack((Result_Rho, result_rho))
        Result_eta = np.vstack((Result_eta, result_eta))
        Result_Mg = np.vstack((Result_Mg, result_mg))

    distance=Result_Rho-obs_den[None,:]
    best_match_index = np.argmin(np.abs(distance), axis=0)
    T_layer=Result_T[best_match_index,np.arange(Result_T.shape[1])]
    Rho_layer=Result_Rho[best_match_index,np.arange(Result_Rho.shape[1])]
    eta_layer=Result_eta[best_match_index,np.arange(Result_eta.shape[1])]
    mg_layer=Result_Mg[best_match_index,np.arange(Result_Mg.shape[1])]
    V_layer=Result_V[best_match_index,np.arange(Result_V.shape[1])]

    
    Result_V_F.extend(V_layer.tolist())
    Result_T_F.extend(T_layer.tolist())
    Result_Rho_F.extend(Rho_layer.tolist())
    Result_eta_F.extend(eta_layer.tolist())
    Result_Mg_F.extend(mg_layer.tolist())

Result_T_F = np.asarray(Result_T_F)
Result_eta_F = np.asarray(Result_eta_F)
Result_Rho_F = np.asarray(Result_Rho_F)
Result_V_F = np.asarray(Result_V_F)
Result_Mg_F = np.asarray(Result_Mg_F)


In [20]:

Out = np.zeros([len(Result_Rho_F), 9])
Out[:, 0:4] = tomo_NA_stack[:,0:4]
Out[:, 4] = Result_T_F
Out[:, 5] = Result_Rho_F
Out[:, 6] = Result_eta_F
Out[:, 7] = Result_V_F
Out[:, 8] = Result_Mg_F
Outs=Out

In [21]:
np.savetxt('ANT20_DTE_priorden.txt', Outs, delimiter=",")

In [22]:
# Inport data
tomo_NA_stack = np.loadtxt('ANT20_grid_PostInversion.txt')

In [23]:
Tmin = 300-273.
Tmax = 3000-273.
dT = 1.      # temperature increment
nT = int((Tmax - Tmin)/dT + 1)
depth=tomo_NA_stack[:,2]
Depths=np.unique(depth)
nDepths = Depths.shape[0]

Mgt = np.arange(89.27,93.97,0.1)
nMg=Mgt.shape[0]

In [24]:
Result_T_F = list()
Result_Rho_F = list()
Result_eta_F = list()
Result_V_F = list()
Result_Mg_F = list()

arr_T = np.linspace(Tmin, Tmax, nT)
for j in range(nDepths):
    Result_T = np.array([]).reshape(0,661*661)
    Result_Rho = np.array([]).reshape(0,661*661)
    Result_eta = np.array([]).reshape(0,661*661)
    Result_V = np.array([]).reshape(0,661*661)
    Result_Mg = np.array([]).reshape(0,661*661)
    arr_select = tomo_NA_stack[:, 2]== Depths[j]
    obs_v = tomo_NA_stack[:, 3][arr_select]/1e3
    obs_den = tomo_NA_stack[:, 4][arr_select]

    for i in range(nMg):
        arr_TVRho = np.zeros([nT, 5])
        arr_TVRho[:, 0] = arr_T
        arr_TVRho[:, 1:] = Vs_calc3(pos,arr_T,-Depths[j]/1e3,Mgt[i])
        syn_T = arr_TVRho[:, 0]
        syn_v = arr_TVRho[:, 1]
        syn_mg = arr_TVRho[:, 3] 
        syn_rho = arr_TVRho[:, 2]  
        syn_eta = arr_TVRho[:, 4]
        ipkwds = dict(left=-1, right=-1)
        result_T = np.interp(obs_v, syn_v[::-1], syn_T[::-1], **ipkwds)
        result_rho = np.interp(obs_v, syn_v[::-1], syn_rho[::-1], **ipkwds)
        result_eta = np.interp(obs_v, syn_v[::-1], syn_eta[::-1], **ipkwds)
        result_v = np.interp(obs_v, syn_v[::-1], syn_v[::-1], **ipkwds)
        result_mg = result_rho - result_rho + Mgt[i]

        Result_V = np.vstack((Result_V, result_v))
        Result_T = np.vstack((Result_T, result_T))
        Result_Rho = np.vstack((Result_Rho, result_rho))
        Result_eta = np.vstack((Result_eta, result_eta))
        Result_Mg = np.vstack((Result_Mg, result_mg))

    distance=Result_Rho-obs_den[None,:]
    best_match_index = np.argmin(np.abs(distance), axis=0)
    T_layer=Result_T[best_match_index,np.arange(Result_T.shape[1])]
    Rho_layer=Result_Rho[best_match_index,np.arange(Result_Rho.shape[1])]
    eta_layer=Result_eta[best_match_index,np.arange(Result_eta.shape[1])]
    mg_layer=Result_Mg[best_match_index,np.arange(Result_Mg.shape[1])]
    V_layer=Result_V[best_match_index,np.arange(Result_V.shape[1])]

    
    Result_V_F.extend(V_layer.tolist())
    Result_T_F.extend(T_layer.tolist())
    Result_Rho_F.extend(Rho_layer.tolist())
    Result_eta_F.extend(eta_layer.tolist())
    Result_Mg_F.extend(mg_layer.tolist())

Result_T_F = np.asarray(Result_T_F)
Result_eta_F = np.asarray(Result_eta_F)
Result_Rho_F = np.asarray(Result_Rho_F)
Result_V_F = np.asarray(Result_V_F)
Result_Mg_F = np.asarray(Result_Mg_F)


In [25]:

Out = np.zeros([len(Result_Rho_F), 9])
Out[:, 0:4] = tomo_NA_stack[:,0:4]
Out[:, 4] = Result_T_F
Out[:, 5] = Result_Rho_F
Out[:, 6] = Result_eta_F
Out[:, 7] = Result_V_F
Out[:, 8] = Result_Mg_F
Outs=Out

In [26]:
np.savetxt('ANT20_DTE_postden.txt', Outs, delimiter=",")